# Deep Reinforcement Lerning Lectures - Policy Gradients

### Imports and auxiliary settings

In [ ]:
!apt update
!apt install -y xvfb x11-utils python3-opengl ffmpeg swig
!pip install swig
!pip install jax==0.4.13 jaxlib==0.4.13
!pip install gymnasium==0.27.1 gymnasium[box2d] pyvirtualdisplay imageio-ffmpeg moviepy==1.0.3
!pip install onnx onnx2pytorch==0.4.1

In [ ]:
# select device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
%matplotlib inline

# PyTorch imports
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.utils.data import Dataset, DataLoader

# Cross Framework library for DL
import onnx
from onnx2pytorch import ConvertModel

# Auxiliary Python imports
import os
import math
import io
import base64
import random
import shutil
import copy
import glob
from time import time, strftime
from tqdm import tqdm
import numpy as np
from tqdm.notebook import tqdm
from time import sleep

# Environment import and set logger level to display error only
# Environment import and set logger level to display error only
import gymnasium as gym
from gymnasium.spaces import Box
from gymnasium import logger as gymlogger
from gymnasium.wrappers import RecordVideo
gymlogger.set_level(gym.logger.ERROR)

# Plotting and notebook imports
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from IPython.display import HTML, display, clear_output

# start virtual display
from pyvirtualdisplay import Display
pydisplay = Display(visible=0, size=(640, 480))
pydisplay.start()

In [ ]:
# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Auxiliary Methods

In [ ]:
class Logger(object):
    """Logger that can be used for debugging different values
    """
    def __init__(self, logdir, params=None, debug=False):
        self.gradients = []
        self.debug = debug
        self.basepath = os.path.join(logdir, strftime("%Y-%m-%dT%H-%M-%S"))
        os.makedirs(self.basepath, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        if params is not None and os.path.exists(params):
            shutil.copyfile(params, os.path.join(self.basepath, "params.pkl"))
        self.log_dict = {}
        self.dump_idx = {}

    def add_gradients(self, grad):
        if not self.debug: return
        self.gradients.append(grad)

    def compute_gradient_variance(self):
        vars_ = []
        grads_list = [np.zeros_like(self.gradients[0])] * 100
        for i, grads in enumerate(self.gradients):
            grads_list.append(grads)
            grads_list = grads_list[1:]
            grad_arr = np.stack(grads_list, axis=0)
            g = np.apply_along_axis(grad_variance, axis=-1, arr=grad_arr)
            vars_.append(np.mean(g))
        return vars_

    @property
    def param_file(self):
        return os.path.join(self.basepath, "params.pkl")

    @property
    def onnx_file(self):
        return os.path.join(self.basepath, "model.onnx")

    @property
    def video_dir(self):
        return os.path.join(self.basepath, "videos")

    @property
    def log_dir(self):
        return os.path.join(self.basepath, "logs")

    def log(self, name, value):
        if name not in self.log_dict:
            self.log_dict[name] = []
            self.dump_idx[name] = -1
        self.log_dict[name].append((len(self.log_dict[name]), time(), value))

    def get_values(self, name):
        if name in self.log_dict:
            return [x[2] for x in self.log_dict[name]]
        return None

    def dump(self):
        for name, rows in self.log_dict.items():
            with open(os.path.join(self.log_dir, name + ".log"), "a") as f:
                for i, row in enumerate(rows):
                    if i > self.dump_idx[name]:
                        f.write(",".join([str(x) for x in row]) + "\n")
                        self.dump_idx[name] = i

In [ ]:
def wrap_env(env, logger, capture_video=True):
    # wrapper for recording
    env = gym.wrappers.RecordEpisodeStatistics(env)
    if capture_video:
        env = gym.wrappers.RecordVideo(env, logger.video_dir, episode_trigger=lambda idx: True)
    return env


def create_env(logger, env_id='BipedalWalker-v3', hardcore=False, capture_video=True):
    # initialize environment
    env = wrap_env(gym.make(env_id, max_episode_steps=400, hardcore=hardcore, render_mode="rgb_array"),
                   logger=logger,
                   capture_video=capture_video)
    action_size = env.action_space.shape[0]
    state_size = env.observation_space.shape[0]
    return env, action_size, state_size


def set_seed(env, seed=None):
    # seeding the envrionment
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)


def transforms(state):
    # transofrm to numpy to tensor and push to device
    return torch.FloatTensor(state).to(device)


def test_environment(env, agent=None, seed=42, n_steps=200):
    # run and evaluate in the environment
    state, info = env.reset(seed=seed)
    for i in range(n_steps):
        env.render()

        if agent is None:
            action = env.action_space.sample()
        else:
            action, _ = agent.act(state)
            action = action.squeeze().cpu().numpy()
        state, reward, done, truncated, info = env.step(action)
        if done:
            state, info = env.reset(seed=seed)
    env.close()


def get_running_stat(stat, stat_len):
    # evaluate stats
    cum_sum = np.cumsum(np.insert(stat, 0, 0))
    return (cum_sum[stat_len:] - cum_sum[:-stat_len]) / stat_len


def plot_results(runner):
    # plot stats
    episode, r, l = np.array(runner.stats_rewards_list).T
    cum_r = get_running_stat(r, 10)
    cum_l = get_running_stat(l, 10)

    plt.figure(figsize=(16, 16))

    plt.subplot(321)

    # plot rewards
    plt.plot(episode[-len(cum_r):], cum_r)
    plt.plot(episode, r, alpha=0.5)
    plt.xlabel('Episode')
    plt.ylabel('Episode Reward')

    plt.subplot(322)

    # plot episode lengths
    plt.plot(episode[-len(cum_l):], cum_l)
    plt.plot(episode, l, alpha=0.5)
    plt.xlabel('Episode')
    plt.ylabel('Episode Length')

    plt.subplot(323)

    # plot return
    all_returns = np.array(runner.buffer.all_returns)
    plt.scatter(range(0, len(all_returns)), all_returns, alpha=0.5)
    mean_returns = np.array(runner.buffer.mean_returns)
    plt.plot(range(0, len(mean_returns)), mean_returns, color="orange")
    plt.xlabel('Episode')
    plt.ylabel('Return')

    plt.subplot(324)

    # plot entropy
    entropy_arr = np.array(runner.stats_entropy_list)
    plt.plot(range(0, len(entropy_arr)), entropy_arr)
    plt.xlabel('Episode')
    plt.ylabel('Entropy')

    plt.subplot(325)

    if runner.logger.debug:
        # plot variance
        variance_arr = np.array(runner.logger.compute_gradient_variance())
        plt.plot(range(0, len(variance_arr)), variance_arr)
        plt.xlabel('Episode')
        plt.ylabel('Variance')

    plt.show()


"""
Utility functions to enable video recording of gym environment and displaying it
"""
def show_video(logger):
    print(logger.video_dir)
    mp4list = glob.glob(f'{logger.video_dir}/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def grad_variance(g):
    # compute gradient variance
    return np.mean(g**2) - np.mean(g)**2

### Test environment

OpenAI offers a set of environments for Reinforcement Learning, which are accessible via the `gymnasium` pip package.
In this exercise we will focus on discrete control tasks using a Box2D simulation known as BipedalWalker-v3.
To access the Box2D packages, we use the gymnasium pypi `box2d` option.

In [ ]:
# show behavior in envrionment with random agent
logger = Logger("logdir")
env, _, _ = create_env(logger)
test_environment(env)
show_video(logger)

### Training Buffer

In [ ]:
class Transition(object):
    """Transition helper object
    """
    def __init__(self, state, action, reward, next_state, log_probs):
        self.state = state
        self.action = action
        self.reward = reward
        self.next_state = next_state
        self.g_return = 0.0
        self.log_probs = log_probs


class Episode(object):
    """Class for collecting an episode of transitions
    """
    def __init__(self, discount):
        self.discount = discount
        self._empty()
        self.total_reward = 0.0

    def _empty(self):
        self.n = 0
        self.transitions = []

    def reset(self):
        self._empty()

    def size(self):
        return self.n

    def append(self, transition):
        self.transitions.append(transition)
        self.n += 1

    def states(self):
        return [s.state for s in self.transitions]

    def actions(self):
        return [a.action for a in self.transitions]

    def rewards(self):
        return [r.reward for r in self.transitions]

    def next_states(self):
        return [s_.next_state for s_ in self.transitions]

    def returns(self):
        return [r.g_return for r in self.transitions]

    def calculate_return(self):
        # calculate the return of the episode
        rewards = self.rewards()
        trajectory_len = len(rewards)
        return_array = torch.zeros((trajectory_len,))
        g_return = 0.
        for i in range(trajectory_len-1, -1, -1):
            g_return = rewards[i] + self.discount*g_return
            return_array[i] = g_return
            self.transitions[i].g_return = g_return
        return return_array


class BufferDataset(Dataset):
    """Buffer dataset used to iterate over buffer samples when training.
    """
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        t = self.data[idx]
        return t.state, t.action, t.reward, t.next_state, t.log_probs


class RolloutBuffer(object):
    # ===================================================
    # ++++++++++++++++++++ OPTIONAL +++++++++++++++++++++
    # ===================================================
    # > feel free to optimize sampling and buffer handling
    # ===================================================
    """Buffer to collect samples while rolling out in the envrionment.
    """
    def __init__(self, capacity, batch_size, min_transitions):
        self.capacity = capacity
        self.batch_size = batch_size
        self.min_transitions = min_transitions
        self.buffer = []
        self._empty()
        self.mean_returns = []
        self.all_returns = []

    def _empty(self):
        # empty the buffer
        del self.buffer[:]
        self.position = 0

    def add(self, episode):
        # Saves a transition
        episode.calculate_return()
        for t in episode.transitions:
            if len(self.buffer) < self.capacity:
                self.buffer.append(None)
            self.buffer[self.position] = t
            self.position = (self.position + 1) % self.capacity

    def update_stats(self):
        # update the statistics on the buffer
        returns = [t.g_return for t in self.buffer]
        self.all_returns += returns
        mean_return = np.mean(np.array(returns))
        self.mean_returns += ([mean_return]*len(returns))

    def reset(self):
        # calls empty
        self._empty()

    def create_dataloader(self):
        # creates a dataloader for training
        train_loader = DataLoader(
            BufferDataset(self.buffer),
            batch_size=self.batch_size,
            shuffle=True
        )
        return train_loader

    def __len__(self):
        return len(self.buffer)

### Define Models

In [ ]:
class ActorNet(nn.Module):
    """Actor network (policy)
    """
    def __init__(self, state_size, action_size, hidden_size):
        super(ActorNet, self).__init__()
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) find a suitable architecture for the actor
        # ===================================================
        # pass
        self.norm = nn.LayerNorm(hidden_size, elementwise_affine=False)
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.mu = nn.Linear(hidden_size, action_size)
        self.sigma = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) compute mu and sigma logit estimates
        # ===================================================
        h = torch.relu(self.norm(self.fc1(x)))
        h = torch.relu(self.norm(self.fc2(h)))
        mu_logits = self.mu(h)
        sigma_logits = self.sigma(h)
        return mu_logits, sigma_logits # must return this!


class CriticNet(nn.Module):
    """Critic network computing the state value
    """
    def __init__(self, state_size, action_size, hidden_size):
        super(CriticNet, self).__init__()
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) find a suitable architecture for the critic
        # ===================================================
        # pass
        self.norm = nn.LayerNorm(hidden_size, elementwise_affine=False)
        self.dense_layer_1 = nn.Linear(state_size, hidden_size)
        self.dense_layer_2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) compute the value estimates
        # ===================================================
        # pass
        h = torch.relu(self.norm(self.dense_layer_1(x)))
        h = torch.relu(self.norm(self.dense_layer_2(h)))

        return self.output(h)


class ActorCriticNet(nn.Module):
    """Combining both networks and add helper methods to act and evaluate samples.
    """
    def __init__(self, state_size, action_size, hidden_size):
        super(ActorCriticNet, self).__init__()
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) initialize your networks
        # ===================================================
        # pass
        self.actor = ActorNet(state_size, action_size, hidden_size)
        self.critic = CriticNet(state_size, action_size, hidden_size)

    def forward(self, x):
        x = x.reshape(1, -1)
        return self.act(x)

    def act(self, state):
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) get mean and sigma logits
        # 2) softplus the sigma logits to ensure only positive values (add some eps value to avoid 0 probabilities)
        # 3) create a Normal distribution based on mu and sigma
        # 4) sample from the distribution
        # 5) get log probabilities
        # ===================================================
        # pass
        mu_logits, sigma_logits = self.actor(state)
        sigma_logits = F.softplus(sigma_logits) + 1e-5
        dist = Normal(mu_logits, sigma_logits)
        action = dist.sample().squeeze() 
        log_probs = dist.log_prob(action)

        return action, log_probs

    def evaluate(self, state, action):
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) get mean and sigma logits
        # 2) softplus the sigma logits to ensure only positive values (add some eps value to avoid 0 probabilities)
        # 3) create a Normal distribution based on mu and sigma
        # 4) get log probabilities
        # 5) get entropy from the distribution
        # 6) evaluate state value
        # ===================================================
        # pass
        mu_logits, sigma_logits = self.actor(state)
        sigma_logits = F.softplus(sigma_logits) + 1e-5
        policy_dist = Normal(mu_logits, sigma_logits)
        log_probs = policy_dist.log_prob(action)
        entropy = policy_dist.entropy()
        state_value = self.critic(state)

        return log_probs, entropy, state_value

### Define Agent

In [ ]:
class Agent(object):
    """Agent class used for training, saving data and handling the model.
    """
    def __init__(self, buffer, state_size, action_size, hidden_size, learning_rate, logger, eps_clip, n_epochs,
                 weight_decay, betas, loss_scales, discount, checkpoint_dir="ckpts"):
        self.action_size = action_size
        self.state_size = state_size
        self.buffer = buffer
        self.checkpoint_dir = checkpoint_dir
        self.loss_scales = loss_scales
        self.n_epochs = n_epochs
        self.eps_clip = eps_clip
        self.logger = logger
        self.discount = discount
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) initialize ActorCriticNet instance
        # 2) create initializers
        # ===================================================
        # pass
        self.model = ActorCriticNet(state_size, action_size, hidden_size).to(device)
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
            betas=betas,
        )


    def save_checkpoint(self, epoch, info=''):
        """Saves a model checkpoint"""
        state = {
            'info': info,
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict()
        }
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        ckp_name = 'best-checkpoint.pth' if info == 'best' else f'checkpoint-epoch{epoch}.pth'
        filename = os.path.join(self.checkpoint_dir, ckp_name)
        torch.save(state, filename)

    def resume_checkpoint(self, resume_path):
        """Resumes training from an existing model checkpoint"""
        print("Loading checkpoint: {} ...".format(resume_path))
        checkpoint = torch.load(resume_path)
        # load architecture params from checkpoint.
        self.model.load_state_dict(checkpoint['state_dict'])
        # load optimizer state from checkpoint only when optimizer type is not changed.
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        print("Checkpoint loaded. Resume training")

    def load_onnx_checkpoint(self, onnx_file):
        self.model.actor = ConvertModel(onnx.load(onnx_file)).to(device)
        self.model.eval()

    def save_onnx_checkpoint(self):
        """Create an ONNX checkpoint"""
        dummy_input = torch.randn((1, self.state_size))
        dummy_input_t = transforms(dummy_input).to(device)
        model = agent.model.actor.to(device)
        torch.onnx.export(model, dummy_input_t, "submission_actor.onnx", verbose=False, opset_version=10, export_params=True, do_constant_folding=True)

    def check(self, file_name):
        # Load the ONNX model
        model = onnx.load(file_name)
        # Check that the IR is well formed
        onnx.checker.check_model(model)

    @torch.no_grad()
    def act(self, state):
        # ===================================================
        # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
        # ===================================================
        # 1) prepare state tensors
        # 2) check if shape is ok or expand acordingly
        # 3) get action and log probabilities
        # ===================================================
        # get action probs then sample from the probabilities
        # pass
        state_tensor = torch.FloatTensor(state).to(device)

        if len(state_tensor.shape) == 1:
            state_tensor = state_tensor.unsqueeze(0)

        action, log_probs = self.model.act(state_tensor)

        return action.cpu(), log_probs

    def train(self):
        for i in range(self.n_epochs):
            # create the a dataloader based on the current buffer
            loader = self.buffer.create_dataloader()
            # iterate over the samples in the dataloader
            for states, actions, rewards, next_states, old_log_probs in loader:
                # ===================================================
                # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
                # ===================================================
                # 1) compute target value with next state and reward
                # 2) compute advantage function from target and current state and action
                # 3) compute importance sampling ratio from log probabilities
                # 4) compute surrogate loss with the advantage and clipped surrogate loss
                # 5) compute value losses
                # 6) compute total loss with entropy regularization
                # 7) compute gradients and perform optimization step
                # ===================================================

                states = states.detach().to(device)
                actions = actions.detach().to(device)
                rewards = rewards.detach().to(device)
                next_states = next_states.detach().to(device)
                old_log_probs = old_log_probs.detach().to(device)

                rewards = rewards.view(-1, 1)

                # normalize rewards
                rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)

                log_probs, entropy, state_values = self.model.evaluate(states, actions)

                target_value = (
                    rewards + self.discount * self.model.critic(next_states).detach()
                )

                advantages = target_value - state_values.detach()

                # normalize advantages
                advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-5)

                ratios = torch.exp(log_probs - old_log_probs.squeeze())

                clipped_ratio = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip)

                policy_loss = -torch.min(
                    ratios * advantages, clipped_ratio * advantages
                ).sum(dim=-1)

                value_loss = F.mse_loss(state_values.float(), target_value.float())

                loss = (
                    self.loss_scales[0] * value_loss
                    + self.loss_scales[1] * policy_loss
                    - self.loss_scales[2] * entropy.view(-1, 1)
                )

                self.optimizer.zero_grad()
                loss.mean().backward()
                self.optimizer.step()

        # return losses and entropy
        return (loss.mean().detach().cpu().numpy(),
                value_loss.mean().detach().cpu().numpy(),
                policy_loss.mean().detach().cpu().numpy()), entropy.mean().detach().cpu().numpy()

### Define Task Runner

In [ ]:
from collections import deque
class Runner(object):
    """Runner class performing the rollout in the environment and calling the agent training function periodically
    """
    def __init__(self, env, agent, buffer, logger, discount=0.99, n_episodes=4000, reward_scale=lambda x: x,
                 use_buffer_reset=True, stats_interval=1, print_stats=True, min_average_reward=300, render=False, seed=42, checkpoint_interval=100, max_episode_steps=2000):
        self.env = env
        self.agent = agent
        self.buffer = buffer
        self.render = render
        self.logger = logger
        self.discount = discount
        self.n_episodes = n_episodes
        self.reward_scale = reward_scale
        self.use_buffer_reset = use_buffer_reset
        self.stats_interval = stats_interval
        self.print_stats = print_stats
        self.min_average_reward = min_average_reward
        self.seed = seed
        self.max_episode_steps = max_episode_steps
        # store stats for plotting
        self.stats_rewards_list = []
        self.stats_entropy_list = []
        # stats for running episodes
        self.timesteps = 0
        self.checkpoint_interval = checkpoint_interval
        self.best_model = None
        self.max_reward = -np.inf

        # Additional param for monitor
        self.number_last_rewards = 100
        self.last_rewads = deque(maxlen=self.number_last_rewards)
        self.all_rewards = []
        self.recorded_episodes = []

    def run(self):
        fig, ax = plt.subplots()
        display_handle = display(fig, display_id=True)
        self.agent.model.eval()
        # train for n episodes
        with tqdm(range(self.n_episodes)) as pbar:
            for e in pbar:
                # reset env and stats
                state, info = self.env.reset(seed=self.seed)
                total_losses = 0.
                value_losses = 0.
                policy_losses = 0.

                # create new episode
                episode = Episode(discount=self.discount)

                # save model
                if e % self.checkpoint_interval == 0:
                    self.agent.save_checkpoint(e)

                done = False
                # train in each episode until episode is done
                while not done:
                    self.timesteps += 1
                    # render env
                    if self.render: self.env.render()

                    # ===================================================
                    # ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
                    # ===================================================
                    # 1) get action and log_probs
                    # 2) step in the environment to collect transitions
                    # 3) uptade agent when buffer has sufficient samples
                    # 4) log statistics and reset buffer
                    # ===================================================

                    # 1) get action and log_probs
                    action, log_probs = self.agent.act(state)

                    # 2) step in the environment to collect transitions
                    # next_state, reward, terminated, truncated, info
                    next_state, reward, terminated, truncated, _ = self.env.step(
                        action.numpy()
                    )
                    done = terminated or truncated

                    episode.total_reward += reward
                    episode.append(
                        Transition(
                            state,
                            action,
                            reward * self.reward_scale,
                            next_state,
                            log_probs,
                        )
                    )

                    # update stats and update agent if done
                    if done:
                        # add current episode to the replay buffer
                        self.buffer.add(episode)
                        self.stats_rewards_list.append(
                            (e, episode.total_reward, episode.size())
                        )

                        # 3) skip if stored episodes are less than the batch size
                        if len(self.buffer) < self.buffer.min_transitions:
                            break

                        (
                            total_loss,
                            value_loss,
                            policy_loss,
                        ), entropy = self.agent.train()
                        total_losses += total_loss
                        value_losses += value_loss
                        policy_losses += policy_loss

                        self.stats_entropy_list.append(entropy)
                        self.buffer.update_stats()

                        if self.use_buffer_reset:
                            self.buffer.reset()

                        if self.print_stats and e % self.stats_interval == 0:
                            score = np.mean(
                                self.stats_rewards_list[-self.stats_interval :],
                                axis=0,
                            )[1]

                            # save episode number for xticks
                            self.last_rewads.append(score)
                            self.all_rewards.append(score)
                            self.recorded_episodes.append(e)

                            pbar.set_description(
                                f"[{e:03d}] Reward: {score:.4f} "
                                f"| Last {self.number_last_rewards}: "
                                f"{int(round(np.min(list(self.last_rewads))))} "
                                f"{int(round(np.mean(list(self.last_rewads))))} "
                                f"{int(round(np.max(list(self.last_rewads))))} "
                            )

                            self.plot_running_rewards(ax)
                            display_handle.update(fig)

                    state = next_state

                # save best model
                if self.best_model is None or episode.total_reward > self.max_reward:
                    self.best_model = copy.deepcopy(self.agent.model)
                    self.agent.save_checkpoint(e, 'best')
                    self.max_reward = episode.total_reward

    @torch.no_grad()
    def plot_running_rewards(self, ax: plt.Axes):
        plt.cla()
        ax.set_title("Training...")

        ax.set_xlabel("Episode")
        ax.set_ylabel("Reward")
        ax.plot(self.recorded_episodes, self.all_rewards, label="Rewards")

        all_rewards = torch.tensor(self.all_rewards, dtype=torch.float)
        means = (
            all_rewards.unfold(
                0, min(all_rewards.shape[0], self.number_last_rewards), 1
            )
            .mean(1)
            .flatten()
        )
        means = torch.cat(
            (
                torch.tensor(
                    [
                        all_rewards[: i + 1].mean()
                        for i in range(
                            min(all_rewards.shape[0] - 1, self.number_last_rewards - 1)
                        )
                    ]
                ),
                means,
            )
        )

        ax.plot(self.recorded_episodes, means.numpy(), label=f"Mean")

        ax.legend(bbox_to_anchor=(1.1, 1.05))
        ax.tick_params(axis="x", labelrotation=90)
        ax.set_xticks(np.arange(0, self.recorded_episodes[-1], 100))

### Train Agent

This is a where you run your actor-critic PPO algorithm.

Some tips as to what you need to implement:
- chose proper architectures for the actor-critic networks
- choose the appropriate loss function (think on which kind of problem you are solving)
- choose an optimizer and its hyper-parameters
- optional: use a learning-rate scheduler
- store your model and training progress often so you don't loose progress if your program crashes

In case you use the provided Logger:
- `logger.log("training_loss", <loss-value>)` to log a particular value
- `logger.dump()` to write the current logs to a log file (e.g. after every episode)
- `logger.log_dir`, `logger.param_file`, `logger.onnx_file`, `logger.video_dir` point to files or directories you can use to save files
- you might want to specify your google drive folder as a logdir in order to automatically sync your results
- if you log the metrics specified in the `plot_metrics` function you can use it to visualize your training progress (or take it as a template to plot your own metrics)

In [ ]:
# create environment
logger = Logger("logdir")
env, action_size, state_size = create_env(logger, capture_video=False)

seed = 31
# set seed
set_seed(env, seed=seed)

# ===================================================
# ++++++++++++++++ YOUR CODE HERE +++++++++++++++++++
# ===================================================
# > find suitable hyperparameters
# hyperparameters
episodes = 2300 # run agent for this many episodes
epochs = 8 # run n epochs of network updates
hidden_size = 128 # number of units in NN hidden layers
learning_rate = 0.0003 # learning rate for optimizer
discount = 0.99 # discount factor gamma value
reward_scale = 1.0 #lambda x: x # reward scaling

batch_size = 128 # number of samples used for an update
min_transitions = 2000 # number of minimum transitions until update is triggered
capacity = 2000 # maximum number of transitions stored in the rollout buffer

use_buffer_reset = True # resets the buffer after every update
eps_clip = 0.2 # clipping or importance sampling updates
loss_scales = (1.0, 0.5, 0.01) # loss scales (value loss, policy loss, entropy loss)
betas = (0.9, 0.999)  # optimizer beta parameters
weight_decay = 0.0001 # optimizer weight decay
checkpoint_interval = 100 # checkpoint interval to overwrite the parameters

# additional settings
print_stats = True
render = False

In [ ]:
buffer = RolloutBuffer(capacity=capacity, batch_size=batch_size, min_transitions=min_transitions)
agent = Agent(buffer=buffer, state_size=state_size, action_size=action_size, hidden_size=hidden_size,
              learning_rate=learning_rate, logger=logger, eps_clip=eps_clip, n_epochs=epochs, weight_decay=weight_decay, betas=betas, loss_scales=loss_scales, discount=discount)
runner = Runner(env=env, agent=agent, buffer=buffer, logger=logger, discount=discount, n_episodes=episodes, reward_scale=reward_scale,
                use_buffer_reset=use_buffer_reset, print_stats=print_stats, render=render, seed=seed)

In [ ]:
runner.run()

In [ ]:
plot_results(runner)

In [ ]:
agent.save_onnx_checkpoint()

## Visualize Agent

In [ ]:
# load model from checkpoint
agent.resume_checkpoint('ckpts/best-checkpoint.pth')

In [ ]:
# run agent in the envrionment
env, _, _ = create_env(logger)
agent.model.eval()
test_environment(env=env, agent=agent)
show_video(logger)

# Demo Evaluations

In [ ]:
agent.load_onnx_checkpoint('submission_actor.onnx')

In [ ]:
# run agent in the envrionment
logger = Logger('logdir')
env, _, _ = create_env(logger)
agent.model.eval()
test_environment(env=env, agent=agent, n_steps=500)
show_video(logger)